In [1]:
from utils import configure_environment
configure_environment()

Seed set to 119182


Device set to cuda


In [2]:
import torch
import numpy as np
import random
from callbacks import LogBinaryDiscoveryData
from models import BinaryConvolutionalClassifier

# Consecutive pattern detection

In [3]:
pattern = [0, 0, 1, 1, 0, 0]
length = 32


# super slow on torch tensors
def pattern_discrimination_data_generator(batch_size=32, low=length, high=length):
    strptr = str(np.array(pattern))[1:-1]
    X = np.random.randint(0, 2, (batch_size, random.randint(low, high)))
    Y = torch.tensor([strptr in str(x) for x in X])
    return torch.tensor(X), Y

In [6]:
model = BinaryConvolutionalClassifier(length, model_scale=.2)
model.fit(pattern_discrimination_data_generator, epochs=2000, callbacks=[LogBinaryDiscoveryData(model, topk=len(pattern), frequency=100)])

1900            accuracy:	0.625            		0 0 1 0 0 0 1 0 0 1 1 0 0 0 0 1 1 0 1 1 1 1 0 1 1 0 1 0 0 0 0 0 
1800            accuracy:	0.6875           		1 1 0 1 1 0 0 0 1 1 1 1 0 1 0 0 0 0 0 1 1 0 0 0 1 0 1 1 1 0 1 1 
1700            accuracy:	0.78125          		1 0 0 0 0 1 0 0 1 0 1 0 1 1 0 0 1 1 1 0 0 1 0 0 1 0 0 0 1 1 0 0 
1600            accuracy:	0.71875          		1 1 0 0 1 1 1 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 0 1 1 1 0 
1500            accuracy:	0.8125           		0 0 1 1 0 1 1 0 0 1 0 0 0 0 0 1 1 0 1 1 1 0 1 1 0 0 0 1 1 0 0 0 
1400            accuracy:	0.9375           		0 0 1 0 1 1 1 0 0 0 1 1 0 0 0 1 0 0 1 0 1 1 1 1 0 0 1 1 0 1 0 0 
1300            accuracy:	1.0              		0 1 1 0 0 1 1 0 0 0 1 0 0 0 1 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 0 0 
1200            accuracy:	1.0              		0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 1 1 1 0 1 
1100            accuracy:	1.0              		0 0 1 1 0 1 0 0 1 1 0 0 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1 0 0 0 0 
1000      

# Subsequence detection

In [16]:
subsequence = [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1]
length = 24


def has_subsequence(sequence):
    i = 0
    for char in sequence:
        i += i < len(subsequence) and char == subsequence[i]
        if i == len(subsequence):
            return True
    return False


# super slow on torch tensors
def subsequence_discrimination_data_generator(batch_size=64, low=length, high=length):
    X = np.random.randint(0, 2, (batch_size, random.randint(low, high)))
    Y = torch.tensor([has_subsequence(x) for x in X])
    return torch.tensor(X), Y


# subsequence_discrimination_data_generator(batch_size=1000)[1].argmax(dim=0).sum()  # aim for about an even label split

In [17]:
model = BinaryConvolutionalClassifier(length, model_scale=2)
model.fit(subsequence_discrimination_data_generator, epochs=10000, callbacks=[LogBinaryDiscoveryData(model, topk=len(subsequence), frequency=100)])

9900            accuracy:	0.734375         		1 1 0 1 0 1 0 1 0 1 0 1 1 1 0 1 0 0 1 0 0 0 0 1 
9800            accuracy:	0.859375         		1 1 1 0 1 1 1 0 0 1 1 0 0 1 1 1 0 0 1 1 0 1 0 0 
9700            accuracy:	0.875            		0 0 1 1 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 1 0 1 0 0 
9600            accuracy:	0.90625          		0 1 1 1 1 1 0 1 0 1 0 0 0 0 1 0 0 1 0 0 1 1 0 1 
9500            accuracy:	0.953125         		0 1 1 1 1 1 0 1 1 1 0 1 0 0 1 1 0 0 0 1 1 1 0 1 
9400            accuracy:	0.984375         		0 0 0 1 0 1 0 0 0 1 0 1 1 1 1 0 1 0 1 1 0 1 1 0 
9300            accuracy:	0.921875         		0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1 1 0 
9200            accuracy:	0.953125         		0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 0 1 0 0 0 0 1 1 1 
9100            accuracy:	0.984375         		1 1 0 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 0 1 0 1 
9000            accuracy:	0.96875          		1 1 0 1 0 1 1 0 1 1 1 0 0 1 0 0 1 1 0 1 0 1 1 1 
8900            accuracy:	0.96875          		1 1 0 0 1 0 0 1